In [2]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [3]:
%sendtofile config.txt

pinled       -2
pinfan       12
boardname    esp8266sense3
connection1  BV6000,bubblino,192.168.43.1


Sent 6 lines (225 bytes) to config.txt.


In [3]:
%sendtofile --source ../Sensor_Kennel/SI7021_funcs.py

Sent 55 lines (2040 bytes) to SI7021_funcs.py.


In [15]:
%sendtofile --source stdmqttas.py
%sendtofile --quiet --source utils.py

Sent 75 lines (2600 bytes) to stdmqttas.py.
Sent 32 lines (969 bytes) to utils.py.


In [13]:
%sendtofile main.py

import time
from machine import Pin
import uasyncio as asyncio
from stdmqttas import fconfig, config, mqttconnecttask, callbackcmdtask, flashledconnectedtask
from stdmqttas import pinled, flashpinled, shortmac, topicstem
from mqtt_as import MQTTClient
from SI7021_funcs import setupSI7021, SI7021checkchip, SI7021printstatus, SI7021humiditytemp

import network
network.WLAN().active(0)  # disable the connection at startup
pinfan = Pin(int(fconfig["pinfan"]), Pin.OUT)
flashpinled(5, 300, 300)

def itertools_count():
    n = 0
    while True:
        yield n
        n += 1

i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4))
print(list(map(hex, i2c.scan())))

topicfan = topicstem+"/fan"
topiccmd = topicstem+"/cmd"
topicreply = topicstem+"/reply"
topiconline = topicstem+"/online"
topicip = topicstem+"/ip"
topicerror = topicstem+"/error"
topicbmeSI7021 = topicstem+'/si7021'

async def si7021task():
    btoinit = True
    for i in itertools_count():
        try:
            if btoinit:
                setupSI7021(i2c)
                btoinit = False
                await asyncio.sleep_ms(100)
            h, t = SI7021humiditytemp()
            payload = "%f %f"%(h, t)
            if client.isconnected():
                print(payload)
                await client.publish(topicbmeSI7021, payload)
        except OSError as e:
            print("si7021task", e)
            btoinit = True
        await asyncio.sleep_ms(500)
        
        
def callbackcmd(topic, msg, retained):
    aloop.create_task(callbackcmdtask(client, topicreply, msg))

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber, "to", client._ssid, client.server)
    await client.publish(topiconline, "1", retain=True)
    await client.publish(topicip, ipnumber, retain=True)
    await client.subscribe(topiccmd, 1)
    
config['port'] = 4500
config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
config['will'] = (topiconline, "0", True)
client = MQTTClient(config)
client.DEBUG = True
    
aloop = asyncio.get_event_loop()
aloop.create_task(si7021task())
aloop.create_task(flashledconnectedtask(client))
aloop.create_task(mqttconnecttask(client))

aloop.run_until_complete(client.publish(topicerror, "", retain=True))
for n in itertools_count():
    try:
        aloop.run_forever()
    except Exception as e:
        aloop.run_until_complete(client.publish(topicerror, str(n)+" "+str(e), retain=True))



Sent 78 lines (2468 bytes) to main.py.


In [11]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [12]:
            h, t = SI7021humiditytemp()


{'wifi_coro': <generator>, 'server': 'mosquitto.doesliverpool.xyz', 'ssl': False, 'port': 0, 'ssl_params': {}, 'response_time': 10, 'will': ('esp8266/1756B9/online', '0', True), 'subs_cb': <function callbackcmd at 0x3fff2640>, 'ssid': 'JulianS9', 'wifi_pw': 'beckaaaa', 'clean_init': True, 'user': '', 'max_repubs': 4, 'password': '', 'ping_interval': 30, 'connect_coro': <generator>, 'clean': True, 'keepalive': 60, 'client_id': b'b9561700'}
